# Datum Sync Demo
Demonstrates the `convert_datum` function which converts between horizontal and vertical CRS using pyproj.

NOTE: An internet connection is required for correct results. Some transformations require pyproj to fetch transformation grids on the fly.

In [1]:
from pyproj import CRS, Transformer

from datum_sync import convert_datum

## Converting from WGS84 3D to Albers+NAVD88: 2 Steps

### Step 1. 3D WGS84 -> NAD83/NAVD88
Convert from 3D WGS84 (https://epsg.io/4979) to NAD83/NAVD88 (https://epsg.io/5498)
These are two geographic coordinate systems using degrees. 4979 elevation is over the WGS84 ellipsoid. This converts horizontally to NAD83 and vertically to NAVD88.

In [2]:
lats = [43.7, 43]
lngs = [-79.4, -79]
zs = [100, 110]
wgs84_3d = 4979  # EPSG
nad83_navd88 = 5498  # EPSG

output_nad83 = convert_datum(xx=lngs, yy=lats, zz=zs, crs_input=wgs84_3d, crs_output=nad83_navd88)

wgs84_coords = [(lngs[i], lats[i], zs[i]) for i in range(len(lats))]
nad83_coords = [
    (output_nad83[0][i], output_nad83[1][i], output_nad83[2][i]) for i in range(len(output_nad83[0]))
]

print(f"Input 3D WGS84 coordinates: {wgs84_coords}")
print(f"Output NAD83/NAVD88 coordinates: {nad83_coords}")

Input 3D WGS84 coordinates: [(-79.4, 43.7, 100), (-79, 43, 110)]
Output NAD83/NAVD88 coordinates: [(-79.39999849691358, 43.69999146739919, 137.63312305578765), (-78.99999685400357, 42.99999183172088, 146.61874386759752)]


### Step 2. NAD83/NAVD88 -> Albers/NAVD88
Now convert the NAD83/NAVD88 to CONUS Albers (horizontal). Albers does not have a vertical component, so it will trigger a warning that z values were not transformed. This is intentional in this case because the coordinates were already transformed from WGS84 height to NAVD88. 

Albers is a projected CRS that uses meters. Now degrees will be in meters.

In [3]:
conus_albers = 5070  # EPSG
nad83_navd88 = 5498  # EPSG

output_albers = convert_datum(
    xx=output_nad83[0],
    yy=output_nad83[1],
    zz=output_nad83[2],
    crs_input=nad83_navd88,
    crs_output=conus_albers,
)

albers_coords = [
    (output_albers[0][i], output_albers[1][i], output_albers[2][i]) for i in range(len(output_albers[0]))
]
print(f"Input NAD83/NAVD88 coordinates: {nad83_coords}")
print(f"Output Albers/NAVD88 coordinates: {albers_coords}")

Input NAD83/NAVD88 coordinates: [(-79.39999849691358, 43.69999146739919, 137.63312305578765), (-78.99999685400357, 42.99999183172088, 146.61874386759752)]
Output Albers/NAVD88 coordinates: [(1325676.0689027791, 2416931.24897092, 137.63312305578765), (1371188.7036548508, 2345672.0544312494, 146.61874386759752)]


/tmp/ipykernel_2154293/2957665717.py:4: ZConversionWarning: Z values were not altered. This could be expected. This may be because input and output CRS do not have vertical element.
  output_albers = convert_datum(


### Step 3. Compare all coordinates
Here shows the transformation from initial 3D WGS84 (degrees, height above ellipsoid) to NAD83/NAVD88 (degrees, height above NAVD88 datum) to Albers/NAVD88 (projected meters, height above NAVD88 datum)

In [4]:
print(f"Input 3D WGS84 coordinates: {wgs84_coords}")
print(f"Input NAD83/NAVD88 coordinates: {nad83_coords}")
print(f"Output Albers/NAVD88 coordinates: {albers_coords}")

Input 3D WGS84 coordinates: [(-79.4, 43.7, 100), (-79, 43, 110)]
Input NAD83/NAVD88 coordinates: [(-79.39999849691358, 43.69999146739919, 137.63312305578765), (-78.99999685400357, 42.99999183172088, 146.61874386759752)]
Output Albers/NAVD88 coordinates: [(1325676.0689027791, 2416931.24897092, 137.63312305578765), (1371188.7036548508, 2345672.0544312494, 146.61874386759752)]


### Example. Run with a pre-constructed transformer. Use a single XY.

In [5]:
conus_albers = CRS.from_epsg(5070)
nad83_navd88 = CRS.from_epsg(5498)
transform = Transformer.from_crs(crs_from=nad83_navd88, crs_to=conus_albers)
xx = -79.4
yy = 43.7

output = convert_datum(xx=-79.4, yy=43.7, transform=transform)

print(f"Input NAD83/NAVD88 coordinates: {(xx, yy)})")
print(f"Output Albers/NAVD88 coordinates: {output[0], output[1]}")

Input NAD83/NAVD88 coordinates: (-79.4, 43.7))
Output Albers/NAVD88 coordinates: (16746853.982679501, 8235388.481230134)
